# Load packages

In [1]:
pip install matplotlib numpy datasets transformers torch scikit-learn tqdm openai seaborn bitsandbytes accelerate\>=0.26.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 47.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.9/680.9 kB 134.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 69.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 333.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 72.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 303.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 216.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 76.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━

# Important: do the following in terminal

In [2]:
#git clone https://github.com/liamdugan/raid.git
#cd raid
#pip install -r requirements.txt

# Hugginface Login (add your login key)

In [2]:
API_KEY="please_replace_with_your_key"

from huggingface_hub import login

# Paste your Hugging Face token when prompted
login(API_KEY)

In [3]:
pip install raid-bench


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 75.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 34.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.2
    Uninstalling numpy-2.1.2:
      Successfully uninstalled numpy-2.1.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
from raid import run_detection, run_evaluation
from raid.utils import load_data

In [7]:
import torch
print(torch.cuda.is_available() and torch.cuda.get_device_capability(0) == (10, 0))

True


# Data Loading 

In [8]:
train_df = load_data(split="train")

100%|██████████| 11.8G/11.8G [03:15<00:00, 60.1MB/s]  


# Experiment for no adverserial attack

In [18]:
train_df_4=train_df[ (train_df['model'] == 'human') & (train_df['attack'] == 'none') ]
train_df_4_sampled = train_df_4.sample(frac=1, random_state=42)
train_df_3=train_df[  (train_df['model'] != 'human')  & (train_df["attack"]=="none") ]# & (train_df['attack'] == 'none') ]#train_df[ (train_df['attack'] == 'synonym') ]# & (train_df['decoding'] == 'sampling') & (train_df['repetition_penalty'] == 'yes') ]
train_df_3_sampled = train_df_3.sample(frac=1, random_state=42)


#(train_df['repetition_penalty'] == 'yes')&

import pandas as pd

human_samples=train_df_4_sampled
train_df_mmini=train_df_3_sampled

In [19]:


import os
import sys
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch.nn.functional as F

os.environ["TOKENIZERS_PARALLELISM"] = "false"

DEVICE = "cuda" if torch.cuda.is_available() else "CPU"


NUM_GPUS = torch.cuda.device_count()

model_name = "tiiuae/falcon-rw-1b"#"EleutherAI/gpt-neo-1.3B"#
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

if NUM_GPUS > 1:
    base_model = torch.nn.DataParallel(base_model)
base_model.to(DEVICE)
base_model.eval()

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

MAX_LENGTH = 512
BATCH_SIZE = 80

# UPDATED my_detector for proper batching (this is critical!)
def my_detector(texts: list[str]) -> list[float]:
    with torch.no_grad():
        tokenized = tokenizer(texts, truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='pt')
        tokenized = {k: v.to(DEVICE) for k, v in tokenized.items()}
        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]

        outputs = base_model(**tokenized)
        logits = outputs.logits[:, :-1, :]
        labels = tokenized["input_ids"][:, 1:]

        # Compute log-likelihood per sample manually
        log_probs = F.log_softmax(logits, dim=-1)
        ll_per_token = log_probs.gather(2, labels.unsqueeze(-1)).squeeze(-1)
        attention_mask = tokenized["attention_mask"][:, 1:]
        ll_per_sample = (ll_per_token * attention_mask).sum(dim=-1) / attention_mask.sum(dim=1).clamp(min=1)

        # Compute entropy per sample
        #neg_entropy = F.softmax(logits, dim=-1) * log_probs
        neg_entropy = (log_probs.exp() * log_probs)

        entropy_per_sample = -(neg_entropy.sum(dim=-1) * attention_mask).sum(-1) / attention_mask.sum(dim=1).clamp(min=1)

        # Combine log-likelihood and entropy
        scores = ( abs(entropy_per_sample+ll_per_sample)).cpu().tolist()#
        

    return scores


def batch_gpu_detector(all_texts):
    results = []
    for i in tqdm(range(0, len(all_texts), BATCH_SIZE), desc="Processing Batches"):
        batch_texts = all_texts[i:i + BATCH_SIZE]
        results.extend(my_detector(batch_texts))
    return results


human_samples=train_df_4_sampled
train_df_mmini=train_df_3_sampled

# Combine human and AI samples
train_df_mini = pd.concat([human_samples, train_df_mmini]).reset_index(drop=True)

# Drop duplicates based on 'id'
train_df_mini = train_df_mini.drop_duplicates(subset="id")




# Run detection
predictions = batch_gpu_detector(train_df_mini['generation'].tolist())

# Prepare evaluation data correctly
results_for_eval = [{"id": row["id"], "score": score} for row, score in zip(train_df_mini.to_dict('records'), predictions)]

# Evaluate
evaluation_result = run_evaluation(results_for_eval, train_df_mini, target_fpr=0.05,require_complete=True, per_domain_tuning=True)
scores_df = pd.DataFrame(evaluation_result['scores'])

# Visualization
#scores_df = pd.DataFrame(evaluation_result['scores'])
#plt.figure(figsize=(12, 6))
#sns.barplot(x='model', y='accuracy', hue='attack', data=scores_df)
#plt.title('Accuracy by Model and Attack Type')
#plt.show()




Processing Batches:   0%|          | 0/5850 [00:00<?, ?it/s]

In [20]:
scores_df.to_csv("scores_output_blackbox.csv", index=False)

# Table for no adverserial attack

In [21]:
scores_df[(scores_df["decoding"]=="all")&(scores_df["model"]=="all")&(scores_df["domain"]=="all") & (scores_df["repetition_penalty"]=="all") ]

,domain,model,decoding,repetition_penalty,attack,tp,fn,accuracy
1580,all,all,all,all,none,375106,79508,0.825109
1673,all,all,all,all,all,375106,79508,0.825109


## Results open source chatmodels (llama-c, mistral-c, mpt-c) - no penalty rep

In [22]:
scores_df[(scores_df["decoding"]=="greedy") & (scores_df["attack"]=="none")
            & ((scores_df["model"]=="llama-chat")|(scores_df["model"]=="mistral-chat")|(scores_df["model"]=="mpt-chat")) 
            & (scores_df["domain"]=="all") & (scores_df["repetition_penalty"]=="no")   ]["accuracy"].mean()

0.9939670431032334

In [23]:
scores_df[(scores_df["decoding"]=="sampling") & (scores_df["attack"]=="none")
            & ((scores_df["model"]=="llama-chat")|(scores_df["model"]=="mistral-chat")|(scores_df["model"]=="mpt-chat")) 
            & (scores_df["domain"]=="all") & (scores_df["repetition_penalty"]=="no")   ]["accuracy"].mean()

0.9789345100092239

## Results open source chatmodels (llama-c, mistral-c, mpt-c) - with penalty rep

In [24]:
scores_df[(scores_df["decoding"]=="greedy") & (scores_df["attack"]=="none")
            & ((scores_df["model"]=="llama-chat")|(scores_df["model"]=="mistral-chat")|(scores_df["model"]=="mpt-chat")) 
            & (scores_df["domain"]=="all") & (scores_df["repetition_penalty"]=="yes")   ]["accuracy"].mean()

0.8638346670655398

In [25]:
scores_df[(scores_df["decoding"]=="sampling") & (scores_df["attack"]=="none")
            & ((scores_df["model"]=="llama-chat")|(scores_df["model"]=="mistral-chat")|(scores_df["model"]=="mpt-chat")) 
            & (scores_df["domain"]=="all") & (scores_df["repetition_penalty"]=="yes")   ]["accuracy"].mean()

0.6219430109939421

## Results open source non-chatmodels (mistral, mpt, gpt2)- no penalty rep

In [26]:
scores_df[(scores_df["decoding"]=="greedy") & (scores_df["attack"]=="none")
            & ((scores_df["model"]=="gpt2")|(scores_df["model"]=="mistral")|(scores_df["model"]=="mpt")) 
            & (scores_df["domain"]=="all") & (scores_df["repetition_penalty"]=="no")   ]["accuracy"].mean()

0.9081345199810534

In [27]:
scores_df[(scores_df["decoding"]=="sampling") & (scores_df["attack"]=="none")
            & ((scores_df["model"]=="gpt2")|(scores_df["model"]=="mistral")|(scores_df["model"]=="mpt")) 
            & (scores_df["domain"]=="all") & (scores_df["repetition_penalty"]=="no")   ]["accuracy"].mean()

0.5399995014085209

## Results open source non-chatmodels (mistral, mpt, gpt2)- with penalty rep

In [28]:
scores_df[(scores_df["decoding"]=="greedy") & (scores_df["attack"]=="none")
            & ((scores_df["model"]=="gpt2")|(scores_df["model"]=="mistral")|(scores_df["model"]=="mpt")) 
            & (scores_df["domain"]=="all") & (scores_df["repetition_penalty"]=="yes")   ]["accuracy"].mean()

0.8402512901054521

In [29]:
scores_df[(scores_df["decoding"]=="sampling") & (scores_df["attack"]=="none")
            & ((scores_df["model"]=="gpt2")|(scores_df["model"]=="mistral")|(scores_df["model"]=="mpt")) 
            & (scores_df["domain"]=="all") & (scores_df["repetition_penalty"]=="yes")   ]["accuracy"].mean()

0.7330291925310997

## Results closed source chatmodels (chatgpt-chat, gpt4, cohere) - no penalty rep

In [30]:
scores_df[(scores_df["decoding"]=="greedy") & (scores_df["attack"]=="none")
            & ((scores_df["model"]=="chatgpt")|(scores_df["model"]=="gpt4")|(scores_df["model"]=="cohere-chat")) 
            & (scores_df["domain"]=="all") & (scores_df["repetition_penalty"]=="no")   ]["accuracy"].mean()

0.9642509909505647

In [31]:
scores_df[(scores_df["decoding"]=="sampling") & (scores_df["attack"]=="none")
            & ((scores_df["model"]=="chatgpt")|(scores_df["model"]=="gpt4")|(scores_df["model"]=="cohere-chat")) 
            & (scores_df["domain"]=="all") & (scores_df["repetition_penalty"]=="no")   ]["accuracy"].mean()

0.7024655348640092

## Results closed source non-chatmodels (cohere, gpt3) - no penalty rep

In [32]:
scores_df[(scores_df["decoding"]=="greedy") & (scores_df["attack"]=="none")
            & ((scores_df["model"]=="cohere")|(scores_df["model"]=="gpt3")) 
            & (scores_df["domain"]=="all") & (scores_df["repetition_penalty"]=="no")   ]["accuracy"].mean()

0.9440954304090943

In [33]:
scores_df[(scores_df["decoding"]=="sampling") & (scores_df["attack"]=="none")
            & ((scores_df["model"]=="cohere")|(scores_df["model"]=="gpt3")) 
            & (scores_df["domain"]=="all") & (scores_df["repetition_penalty"]=="no")   ]["accuracy"].mean()

0.8625383292199535

# Adverserial experiments and results

In [12]:
train_df_4=train_df[ (train_df['model'] == 'human') ]
train_df_4_sampled = train_df_4.sample(frac=1, random_state=42)
train_df_3=train_df[  (train_df['model'] != 'human')  ]# & (train_df['attack'] == 'none') ]#train_df[ (train_df['attack'] == 'synonym') ]# & (train_df['decoding'] == 'sampling') & (train_df['repetition_penalty'] == 'yes') ]
train_df_3_sampled = train_df_3.sample(frac=1, random_state=42)


#(train_df['repetition_penalty'] == 'yes')&

import pandas as pd

human_samples=train_df_4_sampled
train_df_mmini=train_df_3_sampled



In [13]:


import os
import sys
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch.nn.functional as F

os.environ["TOKENIZERS_PARALLELISM"] = "false"

DEVICE = "cuda" if torch.cuda.is_available() else "CPU"


NUM_GPUS = torch.cuda.device_count()

model_name = "tiiuae/falcon-rw-1b"#"EleutherAI/gpt-neo-1.3B"#
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

if NUM_GPUS > 1:
    base_model = torch.nn.DataParallel(base_model)
base_model.to(DEVICE)
base_model.eval()

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

MAX_LENGTH = 512
BATCH_SIZE = 80

# UPDATED my_detector for proper batching (this is critical!)
def my_detector(texts: list[str]) -> list[float]:
    with torch.no_grad():
        tokenized = tokenizer(texts, truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='pt')
        tokenized = {k: v.to(DEVICE) for k, v in tokenized.items()}
        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]

        outputs = base_model(**tokenized)
        logits = outputs.logits[:, :-1, :]
        labels = tokenized["input_ids"][:, 1:]

        # Compute log-likelihood per sample manually
        log_probs = F.log_softmax(logits, dim=-1)
        ll_per_token = log_probs.gather(2, labels.unsqueeze(-1)).squeeze(-1)
        attention_mask = tokenized["attention_mask"][:, 1:]
        ll_per_sample = (ll_per_token * attention_mask).sum(dim=-1) / attention_mask.sum(dim=1).clamp(min=1)

        # Compute entropy per sample
        #neg_entropy = F.softmax(logits, dim=-1) * log_probs
        neg_entropy = (log_probs.exp() * log_probs)

        entropy_per_sample = -(neg_entropy.sum(dim=-1) * attention_mask).sum(-1) / attention_mask.sum(dim=1).clamp(min=1)

        # Combine log-likelihood and entropy
        scores = ( abs(entropy_per_sample+ll_per_sample)).cpu().tolist()#
        

    return scores


def batch_gpu_detector(all_texts):
    results = []
    for i in tqdm(range(0, len(all_texts), BATCH_SIZE), desc="Processing Batches"):
        batch_texts = all_texts[i:i + BATCH_SIZE]
        results.extend(my_detector(batch_texts))
    return results


human_samples=train_df_4_sampled
train_df_mmini=train_df_3_sampled

# Combine human and AI samples
train_df_mini = pd.concat([human_samples, train_df_mmini]).reset_index(drop=True)

# Drop duplicates based on 'id'
train_df_mini = train_df_mini.drop_duplicates(subset="id")




# Run detection
predictions = batch_gpu_detector(train_df_mini['generation'].tolist())

# Prepare evaluation data correctly
results_for_eval = [{"id": row["id"], "score": score} for row, score in zip(train_df_mini.to_dict('records'), predictions)]

# Evaluate
evaluation_result = run_evaluation(results_for_eval, train_df_mini, target_fpr=0.05,require_complete=True, per_domain_tuning=True)
scores_df = pd.DataFrame(evaluation_result['scores'])

# Visualization

#plt.figure(figsize=(12, 6))
#sns.barplot(x='model', y='accuracy', hue='attack', data=scores_df)
#plt.title('Accuracy by Model and Attack Type')
#plt.show()




Processing Batches:   0%|          | 0/70198 [00:00<?, ?it/s]

## Table for adverserial attack

In [14]:
scores_df[(scores_df["decoding"]=="all")&(scores_df["model"]=="all")&(scores_df["domain"]=="all") & (scores_df["repetition_penalty"]=="all") & (scores_df["attack"]!="homoglyph") & (scores_df["attack"]!="all") &(scores_df["attack"]!="zero_width_space")]

,domain,model,decoding,repetition_penalty,attack,tp,fn,accuracy
9764,all,all,all,all,whitespace,276361,178253,0.607903
9857,all,all,all,all,upper_lower,329478,125136,0.724742
9950,all,all,all,all,article_deletion,338343,116271,0.744242
10043,all,all,all,all,none,375106,79508,0.825109
10136,all,all,all,all,alternative_spelling,366566,88048,0.806324
10229,all,all,all,all,perplexity_misspelling,362713,91901,0.797848
10322,all,all,all,all,insert_paragraphs,368468,86146,0.810507
10508,all,all,all,all,number,340254,114360,0.748446
10601,all,all,all,all,paraphrase,337331,117283,0.742016
10694,all,all,all,all,synonym,221025,233589,0.486182


In [17]:
scores_df.to_csv("scores_output_adv.csv", index=False)

# Adverserial with preprocessing if you wish to remove homoglyph and zero width space

In [27]:
train_df_4=train_df[ (train_df['model'] == 'human') ]
train_df_4_sampled = train_df_4.sample(frac=0.05, random_state=42)
train_df_3=train_df[  (train_df['model'] != 'human')  ]# & (train_df['attack'] == 'none') ]#train_df[ (train_df['attack'] == 'synonym') ]# & (train_df['decoding'] == 'sampling') & (train_df['repetition_penalty'] == 'yes') ]
train_df_3_sampled = train_df_3.sample(frac=0.003, random_state=42)


#(train_df['repetition_penalty'] == 'yes')&

import pandas as pd

human_samples=train_df_4_sampled
train_df_mmini=train_df_3_sampled



In [ ]:
import re
import unicodedata

# Extended manual homoglyph map (non-Latin to Latin equivalents)
HOMOGLYPH_MAP = {
    # Cyrillic
    'а': 'a', 'А': 'A', 'в': 'b', 'В': 'B', 'е': 'e', 'Е': 'E',
    'і': 'i', 'І': 'I', 'ј': 'j', 'Ј': 'J', 'к': 'k', 'К': 'K',
    'м': 'm', 'М': 'M', 'н': 'h', 'Н': 'H', 'о': 'o', 'О': 'O',
    'р': 'p', 'Р': 'P', 'с': 'c', 'С': 'C', 'т': 't', 'Т': 'T',
    'у': 'y', 'У': 'Y', 'х': 'x', 'Х': 'X', 'ч': 'ch', 'Ч': 'Ch',
    'ѕ': 's', 'Ѕ': 'S', 'ъ': 'y', 'Ъ': 'Y', 'ь': '', 'Ь': '',
    # Greek
    'α': 'a', 'Α': 'A', 'β': 'b', 'Β': 'B', 'γ': 'g', 'Γ': 'G',
    'ε': 'e', 'Ε': 'E', 'η': 'h', 'Η': 'H', 'ι': 'i', 'Ι': 'I',
    'κ': 'k', 'Κ': 'K', 'ν': 'n', 'Ν': 'N', 'ο': 'o', 'Ο': 'O',
    'ρ': 'p', 'Ρ': 'P', 'σ': 's', 'Σ': 'S', 'τ': 't', 'Τ': 'T',
    'χ': 'x', 'Χ': 'X', 'ζ': 'z', 'Ζ': 'Z',
    # Other confusable characters
    'ꞗ': 'r', 'Ꞗ': 'R', 'ꭓ': 'x', 'ꭔ': 'u', 'ꭕ': 'u', 'ꭖ': 'y',
    'ʍ': 'w', 'ᴡ': 'W', 'ı': 'i', 'ɪ': 'I',
    # Additional scripts (e.g., Armenian, Hebrew)
    'ա': 'w', 'Ա': 'A', 'օ': 'o', 'Օ': 'O', '׳': '\''
}

def clean_text(text: str) -> str:
    """
    Normalize text by removing invisible characters, replacing homoglyphs with Latin equivalents,
    and cleaning up whitespace. Designed to defend against homoglyph spoofing attacks.
    """
    # Step 1: Remove zero-width and invisible characters
    text = re.sub(r'[\u200B-\u200D\uFEFF\u2060\u180E]', '', text)

    # Step 2: Normalize Unicode (e.g., full-width characters, ligatures)
    text = unicodedata.normalize('NFKC', text)

    # Step 3: Replace homoglyphs with Latin equivalents
    text = ''.join(HOMOGLYPH_MAP.get(char, char) for char in text)

    return text

In [24]:

# Apply text preprocessing to neutralize attacks
train_df_3_sampled['generation'] = train_df_3_sampled['generation'].apply(clean_text)
train_df_4_sampled['generation'] = train_df_4_sampled['generation'].apply(clean_text)

In [ ]:


import os
import sys
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch.nn.functional as F

os.environ["TOKENIZERS_PARALLELISM"] = "false"

DEVICE = "cuda" if torch.cuda.is_available() else "CPU"


NUM_GPUS = torch.cuda.device_count()

model_name = "tiiuae/falcon-rw-1b"#"EleutherAI/gpt-neo-1.3B"#
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

if NUM_GPUS > 1:
    base_model = torch.nn.DataParallel(base_model)
base_model.to(DEVICE)
base_model.eval()

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

MAX_LENGTH = 512
BATCH_SIZE = 80

# UPDATED my_detector for proper batching (this is critical!)
def my_detector(texts: list[str]) -> list[float]:
    with torch.no_grad():
        tokenized = tokenizer(texts, truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='pt')
        tokenized = {k: v.to(DEVICE) for k, v in tokenized.items()}
        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]

        outputs = base_model(**tokenized)
        logits = outputs.logits[:, :-1, :]
        labels = tokenized["input_ids"][:, 1:]

        # Compute log-likelihood per sample manually
        log_probs = F.log_softmax(logits, dim=-1)
        ll_per_token = log_probs.gather(2, labels.unsqueeze(-1)).squeeze(-1)
        attention_mask = tokenized["attention_mask"][:, 1:]
        ll_per_sample = (ll_per_token * attention_mask).sum(dim=-1) / attention_mask.sum(dim=1).clamp(min=1)

        # Compute entropy per sample
        #neg_entropy = F.softmax(logits, dim=-1) * log_probs
        neg_entropy = (log_probs.exp() * log_probs)

        entropy_per_sample = -(neg_entropy.sum(dim=-1) * attention_mask).sum(-1) / attention_mask.sum(dim=1).clamp(min=1)

        # Combine log-likelihood and entropy
        scores = ( abs(entropy_per_sample+ll_per_sample)).cpu().tolist()#
        

    return scores


def batch_gpu_detector(all_texts):
    results = []
    for i in tqdm(range(0, len(all_texts), BATCH_SIZE), desc="Processing Batches", file=sys.stdout):
        batch_texts = all_texts[i:i + BATCH_SIZE]
        results.extend(my_detector(batch_texts))
    return results


human_samples=train_df_4_sampled
train_df_mmini=train_df_3_sampled

# Combine human and AI samples
train_df_mini = pd.concat([human_samples, train_df_mmini]).reset_index(drop=True)

# Drop duplicates based on 'id'
train_df_mini = train_df_mini.drop_duplicates(subset="id")




# Run detection
predictions = batch_gpu_detector(train_df_mini['generation'].tolist())

# Prepare evaluation data correctly
results_for_eval = [{"id": row["id"], "score": score} for row, score in zip(train_df_mini.to_dict('records'), predictions)]

# Evaluate
evaluation_result = run_evaluation(results_for_eval, train_df_mini, target_fpr=0.05,require_complete=True, per_domain_tuning=True)


# Visualization
scores_df = pd.DataFrame(evaluation_result['scores'])
plt.figure(figsize=(12, 6))
sns.barplot(x='model', y='accuracy', hue='attack', data=scores_df)
plt.title('Accuracy by Model and Attack Type')
plt.show()




## table for adverserial with preprocessing

In [ ]:
scores_df[(scores_df["decoding"]=="all")&(scores_df["model"]=="all")&(scores_df["domain"]=="all") & (scores_df["repetition_penalty"]=="all") ]